#### Some regular code to check the necessary resources information and connect to google drive to access dataset

In [1]:
# # to check the devices information
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
# # device_name = tf.test.gpu_device_name()
# # if device_name != '/device:GPU:0':
# #   raise SystemError('GPU device not found')
# # print('Found GPU at: {}'.format(device_name))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11648514387832933052
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3665166336
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2784685498216898728
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
]


In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
   raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
# memory footprint support libraries/code
#!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
import psutil

import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 8.1 GB  | Proc size: 698.0 MB
GPU RAM Free: 5799MB | Used: 345MB | Util   6% | Total 6144MB


# Importing libraries

In [4]:
# %tensorflow_version 1.x
import os
# import keras
import json
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from tensorflow.keras import backend as K 
import imageio
from IPython.display import Image

import cv2
import h5py
import tensorflow as tf

# from keras import backend as K
# from keras.engine import Input, Model
from tensorflow import keras
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Conv3D,
    MaxPooling3D,
    concatenate,
    UpSampling3D,
    Dropout,
    Activation,
    Conv3DTranspose,
    Concatenate,
    LayerNormalization,
    #BatchNormalization,
)

from tensorflow_addons.layers import (
    GroupNormalization,
)   
from tensorflow.keras.callbacks import (
    ModelCheckpoint,
    ReduceLROnPlateau,
    CSVLogger,
    EarlyStopping,
    TensorBoard,
    TerminateOnNaN,
)
# from keras.layers.merge import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.compat.v1.logging import INFO, set_verbosity

from contextlib import redirect_stdout

set_verbosity(INFO)

K.set_image_data_format("channels_first")


import random as python_random
np.random.seed(2021)
python_random.seed(2021)
tf.random.set_seed(2021)

print('Tensorflow version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('GPU: ', tf.test.gpu_device_name())

Tensorflow version:  2.6.0
Keras version:  2.6.0
GPU:  /device:GPU:0


## Run external python files

In [5]:
import sys
sys.path.append("D:\Akhilesh_PHD\Brain\BraTS_2020_New1\\import_files")
import visualize
import losses_and_metrics as lnm
import custom_data_generator as data_gen


# Settings

In [7]:
# directory settings
#HOME_DIR = "/content/drive/My Drive/Thesis_Experiment/BraTS_Dataset/"
dir_train = "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Dataset\\image_train"
dir_train_label = "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Dataset\\label_train"
dir_valid = "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Dataset\\image_valid"
dir_valid_label = "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Dataset\\label_valid"


processed_dir = "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1"
checkpoint_path = os.path.join(processed_dir, '\checkpoints\chckpt-1-{epoch:03d}.ckpt')


################################################################################################
# unet model settingsD:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN
dim = (160, 160, 16)
padding = 'same'
activation = 'relu'
data_format = 'channels_first'
#data_format = 'channels_last'
n_channels = 4
kernel_size = (3,3,3)
strides = (1,1,1)
pool_size = (2,2,2)

learning_rate = 0.0001


################################################################################################
# training settings
config_path = "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1"
with open(config_path + "\config_dataset.json") as json_file:
    number_dataset = json.load(json_file)

batch_size = 2
steps_per_epoch = len(number_dataset['train']) // batch_size
n_epochs = 100
validation_steps = len(number_dataset['valid']) // batch_size
reduce_factor = 0.1
patience_reduce_lr = 5
patience_earlystopping = 10
min_lr = 0.0000001


# Exploring the dataset

In [8]:
# function to load the data
def load_case(image_nifty_file, label_nifty_file):
    # load the image and label file, get the image content 
    # and return a numpy array for each
    image = np.array(nib.load(image_nifty_file).get_fdata())
    label = np.array(nib.load(label_nifty_file).get_fdata())
    
    return image, label

def visualize_patch_predict(image, ground_truth, prediction):
	fig, ax = plt.subplots(1, 3, figsize=[10,5], squeeze=False)
	ax[0][0].imshow(image[:, :, 0], cmap='Greys_r')
	ax[0][0].set_title('Image')
	ax[0][0].set_yticks([])
	ax[0][0].set_xticks([])

	ax[0][1].imshow(ground_truth[:, :, 0], cmap='Greys_r')
	ax[0][1].set_title('Ground Truth')
	ax[0][1].set_xticks([])
	ax[0][1].set_yticks([])

	ax[0][2].imshow(prediction[:, :, 0], cmap='Greys_r')
	ax[0][2].set_title('Prediction')
	ax[0][2].set_xticks([])
	ax[0][2].set_yticks([])

	fig.subplots_adjust(wspace=0, hspace=0)
 

# Code to visualize

In [9]:
#image, label = load_case('D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Dataset\\image_train\BRATS_002.nii.gz', 'D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Dataset\\label_train\BRATS_002.nii.gz')
# uncomment below code to see the grid
# image = get_labeled_image(image, label)
# plot_image_grid(image)

# some general information of the loaded image and label
#print('Image_min:', np.min(image), '\t\t\t\t Label_min:', np.min(label))
#print('Image_max:', np.max(image), '\t\t\t Label_max:', np.max(label))
#print('Image_dim:', image.shape, '\t\t Label_dim:', label.shape)
#print('Image_type:', type(image), '\t Label_type:', type(label))
# to visualize the gif
#print('\n\nSagittal \t\t Coronal \t\t Axial')
#visualize.visualize_data_gif(visualize.get_labeled_image(image, label))

# Model

In [10]:
conv_pars = dict(kernel_size=kernel_size, padding=padding, data_format=data_format)
max_pool_pars = dict(pool_size=pool_size, data_format=data_format)

input = Input(shape=(n_channels, dim[0], dim[1], dim[2]))

# down sampling
c1 = Conv3D(32, strides=strides, **conv_pars)(input)
c1 = GroupNormalization(groups=2)(c1)
c1 = Activation(activation) (c1)
c1 = Dropout(0.25)(c1)
c1 = Conv3D(64, strides=strides, **conv_pars)(c1)
c1 = GroupNormalization(groups=2)(c1)
c1 = Activation(activation) (c1)
p1 = MaxPooling3D(**max_pool_pars)(c1)

c2 = Conv3D(64, strides=strides, **conv_pars)(p1)
c2 = GroupNormalization(groups=2)(c2)
c2 = Activation(activation)(c2)
c2 = Dropout(0.25)(c2)
c2 = Conv3D(128, strides=strides, **conv_pars)(c2)
c2 = GroupNormalization(groups=2)(c2)
c2 = Activation(activation)(c2)
p2 = MaxPooling3D(**max_pool_pars)(c2)


c3 = Conv3D(128, strides=strides, **conv_pars)(p2)
c3 = GroupNormalization(groups=2)(c3)
c3 = Activation(activation)(c3)
c3 = Dropout(0.25)(c3)
c3 = Conv3D(256, strides=strides, **conv_pars)(c3)
c3 = GroupNormalization(groups=2)(c3)
c3 = Activation(activation)(c3)
p3 = MaxPooling3D(**max_pool_pars)(c3)


c4 = Conv3D(256, strides=strides, **conv_pars)(p3)
c4 = GroupNormalization(groups=2)(c4)
c4 = Activation(activation)(c4)
c4 = Dropout(0.25)(c4)
c4 = Conv3D(512, strides=strides, **conv_pars)(c4)
c4 = GroupNormalization(groups=2)(c4)
c4 = Activation(activation)(c4)

# # up sampling
u2 = UpSampling3D(size=pool_size, data_format=data_format)(c4)
u2 = concatenate([u2, c3], axis=1)
u2 = Conv3D(256, strides=strides, **conv_pars)(u2)
u2 = GroupNormalization(groups=2)(u2)
u2 = Activation(activation)(u2)
u2 = Dropout(0.25)(u2)
u2 = Conv3D(256, strides=strides, **conv_pars)(u2)
u2 = GroupNormalization(groups=2)(u2)
u2 = Activation(activation)(u2)

u1 = UpSampling3D(size=pool_size, data_format=data_format)(u2)
u1 = concatenate([u1, c2], axis=1)
u1 = Conv3D(128, strides=strides, **conv_pars)(u1)
u1 = GroupNormalization(groups=2)(u1)
u1 = Activation(activation)(u1)
u1 = Dropout(0.25)(u1)
u1 = Conv3D(128, strides=strides, **conv_pars)(u1)
u1 = GroupNormalization(groups=2)(u1)
u1 = Activation(activation)(u1)


u0 = UpSampling3D(size=pool_size, data_format=data_format)(u1)
u0 = concatenate([u0, c1], axis=1)
u0 = Conv3D(64, strides=strides, **conv_pars)(u0)
u0 = GroupNormalization(groups=2)(u0)
u0 = Activation(activation)(u0)
u0 = Dropout(0.25)(u0)
u0 = Conv3D(64, strides=strides, **conv_pars)(u0)
u0 = GroupNormalization(groups=2)(u0)
u0 = Activation(activation)(u0)

output = Conv3D(3, (1,1,1), activation='sigmoid', data_format=data_format)(u0)

model = Model(inputs=input, outputs=output, name=f"UNET-depth4-{data_format}")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), 
              loss=lnm.soft_dice_loss, 
              metrics=[lnm.dice_coefficient])

#model.summary(line_length=135)

# saving the architecture in text file
with open(("D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\\h5_data_B1\\model_summary-1.txt"), 'w') as f:
    with redirect_stdout(f):
        model.summary(line_length=135)

# saving the architecture format in png
keras.utils.plot_model(model, 
                       to_file= "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\\h5_data_B1\\model_image-1.png", 
                       show_shapes=True);

In [11]:
#import json
#with open("D:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN\config_dataset.json") as json_file:
#    config = json.load(json_file)

# Get generators for training and validation sets
#train_generator = data_gen.VolumeDataGenerator(config["train"],"D:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN\train",batch_size=batch_size, dim=dim,verbose=0)
#valid_generator = data_gen.VolumeDataGenerator(config["valid"],"D:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN\valid",batch_size=batch_size, dim=dim,verbose=0)
#checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_dice_coefficient', save_weights_only=True, mode='max', verbose=1)
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=reduce_factor, patience=patience_reduce_lr, min_lr=min_lr, mode='auto')
#csvlogger = CSVLogger(("D:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN\training_log_1.csv"), separator=',', append=False)
#early_stopping = EarlyStopping(monitor='val_loss', patience=patience_earlystopping, verbose=1, mode='auto')
#terminate_on_nan = TerminateOnNaN()

#callbacks = [checkpoint, reduce_lr, csvlogger, early_stopping, terminate_on_nan]

#history = model.fit(x=train_generator, steps_per_epoch=steps_per_epoch, epochs=n_epochs, validation_data=valid_generator, validation_steps=validation_steps, callbacks=callbacks)



In [12]:
with open("D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\config_dataset.json") as json_file:
    config = json.load(json_file)
train_generator = data_gen.VolumeDataGenerator(config["train"],
                                               "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\\train\\",
                                               batch_size=batch_size, dim=dim,
                                               verbose=0)
valid_generator = data_gen.VolumeDataGenerator(config["valid"],
                                               "D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\\valid\\",
                                               batch_size=batch_size, dim=dim,
                                               verbose=0)
checkpoint = ModelCheckpoint(checkpoint_path,
                             monitor='val_dice_coefficient',
                             save_weights_only=True,
                             mode='max',
                             verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=reduce_factor, 
                              patience=patience_reduce_lr,
                              min_lr=min_lr, mode='auto')
csvlogger = CSVLogger(("D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\\training_log_1.csv"),
                      separator=',', append=False)
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=patience_earlystopping, 
                               verbose=1, 
                               mode='auto')
terminate_on_nan = TerminateOnNaN()

callbacks = [checkpoint, reduce_lr, csvlogger, terminate_on_nan]

history = model.fit(x=train_generator, 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=n_epochs, 
                    validation_data=valid_generator, 
                    validation_steps=validation_steps,
                    callbacks=callbacks)


Epoch 1/100


ResourceExhaustedError:  failed to allocate memory
	 [[node UNET-depth4-channels_first/dropout_6/dropout/GreaterEqual (defined at <ipython-input-12-fa8282d60178>:34) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_11646]

Function call stack:
train_function


In [14]:
print('---------------------------------------------------------------------------------')
# to save the model weights explicitly
model.save_weights("D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\model-1_weights_GN_100_1.h5")
#print(f"The model weights has been saved to {"D:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN"} as \'model-1_weights_LN_25.h5\'")
print('---------------------------------------------------------------------------------')

# model into JSON string
json_config = model.to_json()
with open("D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\model-1_to_json_GN_100_1.json", 'w') as json_file:
    json_file.write(json_config)
#print(f"The model JSON configuration has been saved to {"D:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN"} as \'model-1_to_json_LN_25.json\'")
print('---------------------------------------------------------------------------------')

# to save an entire model
model.save("D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\model-1_entire_state_GN_100_1", include_optimizer=True, save_format='tf')
#print(f"The model has been saved to {"D:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN"} as \'model-1_entire_state_LN_25\'.")
print('---------------------------------------------------------------------------------')

model.save("D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\model-1_entire_state_GN_100_1.h5")
#print(f"The model has been saved to {"D:\Akhilesh_PHD\Thesis Experiment\BraTS\h5_dataset_stand_LN"} as \'model-1_entire_state_LN_25\'.")
print('---------------------------------------------------------------------------------')

from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)


---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
INFO:tensorflow:Assets written to: D:\Akhilesh_PHD\Brain\BraTS_2020_New1\Batch_Normalization\h5_data_B1\model-1_entire_state_GN_100_1\assets
---------------------------------------------------------------------------------
---------------------------------------------------------------------------------
now = 2022-08-01 19:29:16.584918
date and time = 01/08/2022 19:29:16
